In [ ]:
# ipynb形式のライブラリのインポート
%run ./lib.ipynb

# 修正したモデルから卒論時に集計したデータを作成する

1. 表

| ベンチマーク名 | 平均誤差率(%) | コスト比(%) |
|---------|----------|---------|


2. 表

| ベンチマーク名 | 採用割合(最大MAPE(%), 最小MAPE(%)) |
|---------|----------------------------|
|         | モデル(1), モデル(2), ...        |


* 平均誤差率：大規模実行時の関数コール回数との比較
* MAPE：トレーニングデータとの比較

In [ ]:
plt.figure(figsize=(4, 3))
plt.xlabel("使用したプロファイル数")
plt.ylabel("平均誤差率(%)")

In [ ]:
# pd.get_option("display.max_columns")
# pd.get_option("display.max_rows")
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

In [ ]:
benchmark = "cg"
fix = "Class"
if benchmark == "bt" or benchmark == "sp":
    processes = processes_onlyBTSP
else:
    processes = processes_excludeBTSP
benchmarkClasses = ["A", "B", "C", "D"]
targetNumOfProcess = 256
fixedBenchmarkClass = "B"
fixedProcess = 64

# 引数の条件に合った生の実験データを取得する
DF = returnRawDFperBenchmark(Benchmark=benchmark, fix=fix, benchmarkClass=benchmarkClasses,
                             FixedProcess=fixedProcess, Processes=processes, FixedBenchmarkClass=fixedBenchmarkClass)
# 取得した生の実験データから NaN が含まれる関数の実験データを削除
noNaNDF = DF.dropna(how='any')
# noNaNDF

returnedCalculatedDF = return_calculatedDF(benchmark=benchmark, noNaNDF=noNaNDF,
                                           targetNumOfProcess=targetNumOfProcess, targetProblemSize=fixedBenchmarkClass, fix=fix)
# returnedCalculatedDF

# numOfData列の要素一覧を作成し、ソートされたリストを、listOfNumDataに格納する
listOfNumOfData = returnedCalculatedDF['numOfData'].tolist()
listOfNumOfData = sorted(list(set(listOfNumOfData)))
# listOfNumOfData


# 使用したプロファイル数をキー・最適モデルでの相対誤差の平均をバリューとした辞書を作成する
dictAverageRelativeErrorOfBestModel = {}
x = []
y = []
for numOfData in listOfNumOfData:
    # 使用したプロファイル数で抽出
    extractedPerNumOfProfileDF = returnedCalculatedDF[returnedCalculatedDF['numOfData'] == numOfData]
    meanDF = extractedPerNumOfProfileDF.mean()
    data = meanDF.at['relativeErrorOfBestModel']
    dictAverageRelativeErrorOfBestModel[numOfData] = data

x = list(dictAverageRelativeErrorOfBestModel.keys())
x
y = [dictAverageRelativeErrorOfBestModel[key] for key in x]
y

plt.figure()
plt.plot(x, y, marker='o')
plt.xlabel("使用したプロファイル数[％]")
plt.ylabel("平均絶対誤差率")

In [ ]:
# benchmarksからbt, spを除外する
benchmarks = [benchmark for benchmark in benchmarks if benchmark !=
              'bt' and benchmark != 'sp']

In [ ]:
dictTmp = returnDictForPlotPerNumOfUsedData(Benchmark=benchmarks, fix="Class", benchmarkClass=[
    "A", "B", "C", "D"], FixedProcess=64, Processes=[1, 2, 4, 8, 16, 32, 64, 128, 256], FixedBenchmarkClass="C")

In [ ]:
tmpDF = pd.DataFrame()
for benchmark in benchmarks:
    listToLearn = [1, 2, 4, 8, 16, 32, 64, 128]
    listToPredict = [256]
    benchmark_x = dictTmp[benchmark]["x"]
    benchmark_y = dictTmp[benchmark]["y"]
    index = benchmark_x.index(len(listToLearn))
    MAPE = benchmark_y[index]
    relativeCost = returnRelativeCost(benchmark=benchmark, variablesToLearn=listToLearn,
                                      variablesToPredict=listToPredict, fixedClassOrProcess="Class", fixed="C")
    dictRowData = {"ベンチマーク名": benchmark, "平均絶対誤差率": int(10*MAPE)/10, "相対コスト": int(10*relativeCost)/10}
    iDF = pd.DataFrame.from_dict(dictRowData, orient='index').T
    tmpDF = tmpDF.append(iDF)
print(tmpDF.to_latex())

In [ ]:
# dictTmp

plt.figure()
for benchmark in list(dictTmp.keys()):
    x = dictTmp[benchmark]["x"]
    y = dictTmp[benchmark]["y"]
    plt.plot(x, y, marker='o', label=benchmark.upper())
    plt.legend()
    plt.xlabel("使用したプロファイル数")
    plt.ylabel("平均絶対誤差率[％]")

In [ ]:
# 実際にプロットする


# print(f"fix={fix}, benchmarkClasses={benchmarkClasses}, fixedProcess={fixedProcess}, Processes={processes}, FixedBenchmarkClass={fixedBenchmarkClass}")
# print(f"targetNumOfProcess={targetNumOfProcess}, targetProblemSize={fixedBenchmarkClass}, fix={fix}")

# DF = returnRawDFperBenchmark(Benchmark="mg", fix="Process", benchmarkClass=["A", "B", "C", "D"], Processes=[
#                              1, 2, 4, 8, 16, 32, 64, 128, 256], FixedBenchmarkClass="B", FixedProcess=64)
# DF.dropna(how='any')
# DF